# Welter



Monday, May 4, 2016  

## Phase of variability.

part 1

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

In [ ]:
% matplotlib inline
% config InlineBackend.figure_format = 'retina'
import seaborn as sns
sns.set_context('paper', font_scale=1.4)
sns.set_style('ticks')

## ASASSN Data from Subo Dong

In [ ]:
#! head ../data/photometry/LkCa4.dat

In [ ]:
import pandas as pd

The header contains 3 extraneous octothorpes "###".  Read the data just to get the columns.

In [ ]:
fn = '../data/photometry/LkCa4.dat'
names = pd.read_csv(fn, delim_whitespace=True, nrows=0).columns[1:]
dat = pd.read_csv(fn, delim_whitespace=True, names=names, header=0, index_col=False)
#dat.head()

In [ ]:
plt.plot(dat.JD, dat.mag, '.')
plt.ylim(13.5, 12.2)
plt.xlim(2457310)#, 2457390)

plt.vlines(2457341.0, 12.2, 13.5, linestyles='dotted')

#plt.plot(x, y)

## Model

In [ ]:
def lnlike(theta, x, y, yerr):
    A, b0, B, p, lnf = theta
    model = b0 + A*np.sin(2.0*np.pi*(x-B)/p) # + B*np.cos(2.0*np.pi*x/p)
    inv_sigma2 = 1.0/(yerr**2 + model**2*np.exp(2*lnf))
    return -0.5*(np.sum((y-model)**2*inv_sigma2 - np.log(inv_sigma2)))

In [ ]:
def lnprior(theta):
    A, b0, B, p, lnf = theta
    if (3.3 < p < 3.4) and (0.35 < A < 0.9) and (0 < B < p):
        return 0.0
    return -np.inf

In [ ]:
def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr)

In [ ]:
guess = np.array([0.80,12.9, 2.0, 3.375, -4.0])

In [ ]:
ndim, nwalkers = 5, 40
pos = [guess + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

In [ ]:
inds = (dat.JD > 2457060) 

In [ ]:
x = dat.JD[inds]
y = dat.mag[inds]
yerr = dat.mag_err[inds]

In [ ]:
import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y, yerr))

In [ ]:
n_samples = 5000

In [ ]:
output = sampler.run_mcmc(pos, n_samples)

In [ ]:
ws = sampler.chain

In [ ]:
ws.shape

In [ ]:
from matplotlib.ticker import MaxNLocator

In [ ]:
fig, axes = plt.subplots(5, 1, sharex=True, figsize=(8, 14))
for i in range(0, 5, 1):
    axes[i].plot(ws[:, :, i].T, color="k", alpha=0.2)
    axes[i].yaxis.set_major_locator(MaxNLocator(5))
    #axes[i].set_ylabel(label[i])

#axes[13].set_xlabel("step number")

fig.tight_layout(h_pad=0.0)

In [ ]:
sampler.chain.shape

In [ ]:
samples = sampler.chain[:, 3000:, :].reshape((-1, ndim))

In [ ]:
import corner

In [ ]:
fig = corner.corner(samples, labels=["$A$ (mag)", "$b0$ (mag)", "$B$ (mag)", "$p$ (days)", "$\ln{f}$"])
#fig.savefig("triangle.png")

In [ ]:
samples.shape

In [ ]:
A, b0, B, p, lnf = samples[10000]

In [ ]:
x_dense = 2457313.88562 + np.arange(0, 300, 0.1)

In [ ]:
x_dense.shape

In [ ]:
model = b0 + A*np.sin(2.0*np.pi*(x_dense-B)/p) # + B*np.cos(2.0*np.pi*x/p)

In [ ]:
plt.figure(figsize=(12, 10))
plt.plot(dat.JD, dat.mag, '.')
plt.ylim(13.5, 12.2)
plt.xlim(2457310,2457310+150)#, 2457390)

plt.vlines(2457341.0, 12.2, 13.5, linestyles='dotted')

plt.plot(x_dense, model)

In [ ]:
p = 3.374

In [ ]:
phased = np.mod(x, p)

In [ ]:
phased_IG = np.mod(2457344.8609722229, p)
phased_IG2 = np.mod(2456990.790381945, p)

In [ ]:
y.shape

In [ ]:
dat.UT_date[inds].values[0], dat.UT_date[inds].values[-1]

In [ ]:
sns.set_context('talk')

In [ ]:
phased_IG2

In [ ]:
sns.set_context('poster')

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(phased/p, y, 'k.')
plt.plot(phased/p+1.0, y, 'k.')
plt.vlines(phased_IG/p, 12.51, 13.29, linestyles='dotted')
#plt.vlines(1.0+phased_IG2/p, 12.51, 13.29, linestyles='dashed')
plt.xlabel('Phase')
plt.ylabel('mag')
plt.title('LkCa4 in ASASSN 2015-2016')
plt.xlim(0.35, 1.35)
plt.ylim(13.3, 12.5)
#plt.savefig('../results/fig/phase_folded_lightcurve.pdf')
plt.savefig('../results/coolstars19/ASASSN_phase_poster.pdf', bbox_inches="tight")

In [ ]:
from astroML.time_series import multiterm_periodogram
from astroML.time_series import lomb_scargle

In [ ]:
periods = np.linspace(3.35, 3.40, 100)

omega = 2.00*np.pi/periods

P_M = multiterm_periodogram(x, y, yerr, omega)

In [ ]:
plt.step(periods, P_M, label='Multi-term periodogram')
plt.step(periods, P_LS, label='Lomb Scargle')
plt.legend()

In [ ]:
P_LS = lomb_scargle(x, y, yerr, omega)

In [ ]:
np.argmax(P_LS), np.argmax(P_M)

In [ ]:
periods[48]

In [ ]:
import astroML.time_series

In [ ]:
mtf = astroML.time_series.MultiTermFit(3.374, 4)

In [ ]:
mtf_fit = mtf.fit(x, y, yerr)

In [ ]:
mtf_fit.

In [ ]:
phz, yfit = mtf.predict(100)

The end.